# 03 NLP on Reviews

In [1]:
import numpy as np
import pandas as pd

In [2]:
#import data
raw_data = pd.read_csv('../Yelp-NLP-Recommender/data/restaurant_reviews_las_vegas.csv')

In [3]:
raw_data.head()

business_id                       name neighborhood  \
0  Pd52CjgyEU3Rb8co6QfTPw  "Flight Deck Bar & Grill"    Southeast   
1  Pd52CjgyEU3Rb8co6QfTPw  "Flight Deck Bar & Grill"    Southeast   
2  Pd52CjgyEU3Rb8co6QfTPw  "Flight Deck Bar & Grill"    Southeast   
3  Pd52CjgyEU3Rb8co6QfTPw  "Flight Deck Bar & Grill"    Southeast   
4  Pd52CjgyEU3Rb8co6QfTPw  "Flight Deck Bar & Grill"    Southeast   

                   address       city state  postal_code   latitude  \
0  "6730 S Las Vegas Blvd"  Las Vegas    NV      89119.0  36.066914   
1  "6730 S Las Vegas Blvd"  Las Vegas    NV      89119.0  36.066914   
2  "6730 S Las Vegas Blvd"  Las Vegas    NV      89119.0  36.066914   
3  "6730 S Las Vegas Blvd"  Las Vegas    NV      89119.0  36.066914   
4  "6730 S Las Vegas Blvd"  Las Vegas    NV      89119.0  36.066914   

    longitude  stars_x  ...   is_open  \
0 -115.170848      4.0  ...       1.0   
1 -115.170848      4.0  ...       1.0   
2 -115.170848      4.0  ...       1.0   
3 -115.170848      4.0  ...       1.0   
4 -115.170848      4.0  ...       1.0   

                                          categories               review_id  \
0  Nightlife;Bars;Barbeque;Sports Bars;American (...  aG4aEOW86hut943oqL_DOg   
1  Nightlife;Bars;Barbeque;Sports Bars;American (...  fQpivmd02GA4eZJRZGgDQQ   
2  Nightlife;Bars;Barbeque;Sports Bars;American (...  3recIhisA9vC4wnQOMCj7g   
3  Nightlife;Bars;Barbeque;Sports Bars;American (...  18i3YtCuEGMxYSS7fQdR8Q   
4  Nightlife;Bars;Barbeque;Sports Bars;American (...  brbrSd8nHkAEoN56ADEl9A   

                  user_id stars_y        date  \
0  9kV4tmBcE-l5tJqzgKjZRA     4.0  2017-03-27   
1  i7Z7iiDSbWq-npAYHQUeew     5.0  2016-08-27   
2  7Iga45IMB_VKyPZnbhSE2w     1.0  2016-01-09   
3  IvBzaJeCcuChLuKGq3xhxA     2.0  2016-08-22   
4  nIV1-sMoRFrnEtO1TUfLZQ     1.0  2015-10-28   

                                                text useful  funny  cool  
0  Nice, open bar with great views of the strip a...    0.0    0.0   1.0  
1  We usually stop here after using the driving r...    0.0    0.0   0.0  
2  The first time we ate here everything was good...    1.0    1.0   0.0  
3  So I took my daughter there for breakfast and ...    1.0    0.0   0.0  
4  This restaurant has a new name it used to be c...    1.0    0.0   0.0  

[5 rows x 21 columns]

In [33]:
text = raw_data['text']
ratings = raw_data['stars_y']

In [5]:
text.shape

(929607,)

In [6]:
text.head()

0    Nice, open bar with great views of the strip a...
1    We usually stop here after using the driving r...
2    The first time we ate here everything was good...
3    So I took my daughter there for breakfast and ...
4    This restaurant has a new name it used to be c...
Name: text, dtype: object

In [1]:
#NLP Processing
import nltk
#import spacy

#nlp = spacy.load('en_core_web_sm')

In [11]:
text[0]

'Nice, open bar with great views of the strip and golf course.  Patio seating also.  Nice happy hour prices.  Had the carnitas nachos...very good.  And they serve breakfast all day.  Not noisy or smoky'

In [13]:
#doc = nlp(text[0])
#doc

In [19]:
#An Example of what spaCy can do
#for token in doc:
#    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
#          token.shape_, token.is_alpha, token.is_stop)

Nice nice ADJ JJ amod Xxxx True False
, , PUNCT , punct , False False
open open ADJ JJ amod xxxx True False
bar bar NOUN NN ROOT xxx True False
with with ADP IN prep xxxx True True
great great ADJ JJ amod xxxx True False
views view NOUN NNS pobj xxxx True False
of of ADP IN prep xx True True
the the DET DT det xxx True True
strip strip NOUN NN pobj xxxx True False
and and CCONJ CC cc xxx True True
golf golf NOUN NN compound xxxx True False
course course NOUN NN conj xxxx True False
. . PUNCT . punct . False False
    SPACE     False False
Patio patio PROPN NNP nsubj Xxxxx True False
seating seat VERB VBG ROOT xxxx True False
also also ADV RB advmod xxxx True True
. . PUNCT . punct . False False
    SPACE     False False
Nice nice PROPN NNP amod Xxxx True False
happy happy ADJ JJ amod xxxx True False
hour hour NOUN NN compound xxxx True False
prices price NOUN NNS ROOT xxxx True False
. . PUNCT . punct . False False
    SPACE     False False
Had have VERB VBD ROOT Xxx True False
the the

In [26]:
#from spacy import displacy
#
#displacy.render(doc,style='ent', jupyter=True)

## Need to narrow down the possible choices for word vectorization (converting words to probability matrices based on context and neighbors).

### Possible Options ###
Genism: https://radimrehurek.com/gensim/ <br>
fastText: https://fasttext.cc/ <br>
word2vec: https://code.google.com/archive/p/word2vec/ <br>
sklearn: https://scikit-learn.org/stable/modules/classes.html#module-sklearn.feature_extraction.text



### My first choice is sklearn.
Example of two sentence NLP with sklearn's CountVectorizer, term frequency, and nltk's word vocabulary.

This process is similar to that of http://www.pitt.edu/~naraehan/presentation/Movie+Reviews+sentiment+analysis+with+Scikit-Learn.html

In [26]:
from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\crazy99ta\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [10]:
# Initialize a CoutVectorizer to use NLTK's tokenizer instead of its 
# default one (which ignores punctuation and stopwords). 
# Minimum document frequency set to 1. 
foovec = CountVectorizer(min_df=1, tokenizer=nltk.word_tokenize)

In [18]:
# First two reviews.
sents = [text[0], text[1]]
sents

['Nice, open bar with great views of the strip and golf course.  Patio seating also.  Nice happy hour prices.  Had the carnitas nachos...very good.  And they serve breakfast all day.  Not noisy or smoky',
 "We usually stop here after using the driving range. The place is very clean,including the restrooms. The food is good and the employees are friendly. The atmosphere is casual and relaxing. The view of the Strip is a plus.There are numerous TV's to watch sports.\r\nThis would be a great place to watch the football games after hitting a bucket of balls."]

In [24]:
%pprint

Pretty printing has been turned OFF


In [25]:
# sents turned into sparse vector of word frequency counts
sents_counts = foovec.fit_transform(sents)
# foovec now contains vocab dictionary which maps unique words to indexes
foovec.vocabulary_

{'nice': 38, ',': 1, 'open': 43, 'bar': 12, 'with': 70, 'great': 29, 'views': 67, 'of': 42, 'the': 58, 'strip': 57, 'and': 8, 'golf': 27, 'course': 19, '.': 2, 'patio': 45, 'seating': 52, 'also': 7, 'happy': 31, 'hour': 34, 'prices': 48, 'had': 30, 'carnitas': 16, 'nachos': 37, '...': 3, 'very': 65, 'good': 28, 'they': 59, 'serve': 53, 'breakfast': 14, 'all': 6, 'day': 20, 'not': 40, 'noisy': 39, 'or': 44, 'smoky': 54, 'we': 69, 'usually': 64, 'stop': 56, 'here': 32, 'after': 5, 'using': 63, 'driving': 21, 'range': 49, 'place': 46, 'is': 36, 'clean': 18, 'including': 35, 'restrooms': 51, 'food': 23, 'employees': 22, 'are': 9, 'friendly': 25, 'atmosphere': 10, 'casual': 17, 'relaxing': 50, 'view': 66, 'a': 4, 'plus.there': 47, 'numerous': 41, 'tv': 62, "'s": 0, 'to': 61, 'watch': 68, 'sports': 55, 'this': 60, 'would': 71, 'be': 13, 'football': 24, 'games': 26, 'hitting': 33, 'bucket': 15, 'balls': 11}

In [28]:
# Two sentences with 72 unique words
sents_counts.shape

(2, 72)

In [29]:
sents_counts.toarray()

array([[0, 1, 5, 1, 0, 0, 1, 1, 2, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0,
        0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 2, 1, 1, 0, 1, 1,
        1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 2, 1, 0, 0, 0, 0, 0, 1,
        0, 1, 0, 0, 1, 0],
       [1, 1, 6, 0, 3, 2, 0, 0, 2, 2, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1,
        1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 4, 0, 0, 0, 0, 1, 2, 0,
        0, 0, 2, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 9, 0, 1, 2, 1, 1, 1, 1,
        1, 0, 2, 1, 0, 1]], dtype=int64)

In [30]:
# Convert raw frequency counts into TF-IDF (Term Frequency -- Inverse Document Frequency) values
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
sents_tfidf = tfidf_transformer.fit_transform(sents_counts)

In [31]:
# TF-IDF values
# raw counts have been normalized against document length, 
# terms that are found across many docs are weighted down
sents_tfidf.toarray()

array([[0.        , 0.1019111 , 0.50955552, 0.1432325 , 0.        ,
        0.        , 0.1432325 , 0.1432325 , 0.20382221, 0.        ,
        0.        , 0.        , 0.1432325 , 0.        , 0.1432325 ,
        0.        , 0.1432325 , 0.        , 0.        , 0.1432325 ,
        0.1432325 , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.1432325 , 0.1019111 , 0.1019111 ,
        0.1432325 , 0.1432325 , 0.        , 0.        , 0.1432325 ,
        0.        , 0.        , 0.1432325 , 0.286465  , 0.1432325 ,
        0.1432325 , 0.        , 0.1019111 , 0.1432325 , 0.1432325 ,
        0.1432325 , 0.        , 0.        , 0.1432325 , 0.        ,
        0.        , 0.        , 0.1432325 , 0.1432325 , 0.1432325 ,
        0.        , 0.        , 0.1019111 , 0.20382221, 0.1432325 ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.1019111 , 0.        , 0.1432325 , 0.        , 0.        ,
        0.1432325 , 0.        ],
       [0.08427

In [34]:
print('With the following ratings, we should be able to perform ML on the matrix above.')
print("First sentence's rating is ", ratings[0], ".")
print("Second sentence's rating is ", ratings[1], ".")

With the following ratings, we should be able to perform ML on the matrix above.
First sentence's rating is  4.0 .
Second sentence's rating is  5.0 .


Process all 929607 reviews the same way.

In [45]:
text_array = np.asarray(text)

In [46]:
text.shape

(929607,)

In [69]:
# Choose 100k reviews
n=100000
text_small = text[:100000]   #for the end project, randomize
ratings_subset = ratings[:n]

In [60]:
type(text_small)

<class 'pandas.core.series.Series'>

In [61]:
text_vec = CountVectorizer(min_df=2, tokenizer=nltk.word_tokenize)         # use all words
# test_vec = CountVectorizer(min_df=2, tokenizer=nltk.word_tokenize, max_features = 3000) # use top 3000 words only

text_counts = text_vec.fit_transform(text_small.values.astype('U'))

In [62]:
#42582 unique words
text_counts.shape

(100000, 42582)

In [63]:
# Convert raw frequency counts into TF-IDF values
text_tfidf = tfidf_transformer.fit_transform(text_counts)

In [64]:
type(text_tfidf)

<class 'scipy.sparse.csr.csr_matrix'>

In [65]:
from scipy import sparse

In [73]:
#Save for use in next sections
sparse.save_npz('./data/reviews_subset_tfidf.npz', matrix=text_tfidf, compressed=True)

In [70]:
#Save ratings
type(ratings)

<class 'pandas.core.series.Series'>

In [72]:
import pandas as pd

In [75]:
ratings_subset.to_csv(path='./data/ratings_subset.csv')